In [4]:





############ Open Trading WorkStation before running this code to ensure connection ############






In [5]:
#https://www.interactivebrokers.com/campus/ibkr-quant-news/ib_insync-guide-interactive-brokers-api/

#Note that ib_insync was not created by the IB team.

In [1]:
#We create an instance of IB and use it to connect to the local TWS application at the specified host and port. 
#The clientId is a unique identifier for your connection, allowing multiple connections from the same machine.

#The util.startLoop() function enables the asynchronous event loop required by ib_insync when being run in Jupyter Notebook.
#TWS is Trader WorkStation

from ib_insync import *
util.startLoop()

ib = IB()
#ib.connect('127.0.0.1', 7497, clientId=12)
ib.connect()


<IB connected to 127.0.0.1:7497 clientId=1>

Error 10089, reqId 4: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.Delayed market data is available.NVDA NASDAQ.NMS/TOP/ALL, contract: Stock(conId=4815747, symbol='NVDA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='NVDA', tradingClass='NMS')
Error 10089, reqId 6: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.Delayed market data is available.NVDA NASDAQ.NMS/TOP/ALL, contract: Stock(conId=4815747, symbol='NVDA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='NVDA', tradingClass='NMS')
Error 10089, reqId 10: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.Delayed market data is available.NVDA NASDAQ.NMS/TOP/ALL, contract: Stock(conId=4815747, symbol='NVDA', exchange='SMART', primaryExchange='NASDAQ',

In [15]:
nvda = Stock("NVDA", "SMART", "USD")
ib.qualifyContracts(nvda)
[ticker] = ib.reqTickers(nvda)
nvdaValue = ticker.marketPrice()

In [9]:
chains = ib.reqSecDefOptParams(nvda.symbol, "", nvda.secType, nvda.conId)
chain = next(c for c in chains if c.tradingClass == "NVDA" and c.exchange == "SMART")

strikes = [
    strike for strike in chain.strikes
    if strike % 5 == 0
    and nvdaValue - 20 < strike < nvdaValue + 20
]

expirations = sorted(exp for exp in chain.expirations)[:3]
rights = ["C", "P"]

In [14]:
nvdaValue

nan

In [6]:
contracts = [
    Option("NVDA", expiration, strike, right, "SMART", tradingClass="NVDA")
    for right in rights
    for expiration in expirations
    for strike in strikes
]

contracts = ib.qualifyContracts(*contracts)
tickers = ib.reqTickers(*contracts)

contractData = [
    (
        t.contract.lastTradeDateOrContractMonth, 
        t.contract.strike, 
        t.contract.right,
        t.time,
        t.close,
        nvdaValue,
    )
    for t
    in tickers
]

fields = [
    "expiration", 
    "strike", 
    "right", 
    "time", 
    "undPrice",
    "close", 
]

util.df([dict(zip(fields, t)) for t in contractData])

In [8]:
util.df

<function ib_insync.util.df(objs, labels: Optional[List[str]] = None)>